In [9]:
from sklearn.ensemble import VotingClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector

In [7]:
cancer = pd.read_csv(r"C:\Training\Academy\Statistics (Python)\Cases\Wisconsin\BreastCancer.csv",index_col=0)
X = cancer.drop('Class', axis=1)
y = cancer['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24, test_size=0.3, stratify=y)

In [27]:
dtc = DecisionTreeClassifier(random_state=24, max_depth=3)
lr = LogisticRegression(random_state=24, C=0.4)
nb = GaussianNB()
voting = VotingClassifier([('DT',dtc), ('LR',lr),('NB',nb)], voting='soft') 
voting.fit(X_train, y_train)
y_pred = voting.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9761904761904762


In [29]:
y_pred_prob = voting.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test, y_pred_prob))

0.9915458937198067


### Grid Search CV

In [34]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
params = {'DT__max_depth':[None, 2, 3], 'DT__min_samples_split':[2,10,20], 
          'DT__min_samples_leaf':[1,10,20],'LR__C':np.linspace(0.001, 3, 5)}
gcv = GridSearchCV(voting, param_grid=params, cv=kfold, scoring='roc_auc')
gcv.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=VotingClassifier(estimators=[('DT',
                                                     DecisionTreeClassifier(max_depth=3,
                                                                            random_state=24)),
                                                    ('LR',
                                                     LogisticRegression(C=0.4,
                                                                        random_state=24)),
                                                    ('NB', GaussianNB())],
                                        voting='soft'),
             param_grid={'DT__max_depth': [None, 2, 3],
                         'DT__min_samples_leaf': [1, 10, 20],
                         'DT__min_samples_split': [2, 10, 20],
                         'LR__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00])},
             scoring='roc_auc')

In [36]:
print(gcv.best_params_)
print(gcv.best_score_)

{'DT__max_depth': None, 'DT__min_samples_leaf': 1, 'DT__min_samples_split': 2, 'LR__C': 0.001}
0.9937986416496842


## HR Analytics

In [41]:
import os 
os.chdir(r"C:\Training\Academy\Statistics (Python)\Cases\human-resources-analytics")
hr = pd.read_csv("HR_comma_sep.csv")
X = hr.drop('left', axis=1)
y = hr['left']
ohe = OneHotEncoder(handle_unknown='ignore')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False)
dtc = DecisionTreeClassifier(random_state=24, max_depth=3)
lr = LogisticRegression(random_state=24, C=0.4)
nb = GaussianNB()
voting = VotingClassifier([('DT',dtc), ('LR',lr),('NB',nb)], voting='soft') 
pipe = Pipeline([('CT',ct),('VOTING',voting)])
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001E9C238FE90>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001E9C133CE90>)],
                     verbose_feature_names_out=False)),
  ('VOTING',
   VotingClassifier(estimators=[('DT',
                                 DecisionTreeClassifier(max_depth=3,
                                                        random_state=24)),
                                ('LR', LogisticRegression(C=0.4, random_state=24)),
                                ('NB', GaussianNB())],
                    voting='soft'))],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',

In [43]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
params = {'VOTING__DT__max_depth':[None, 2, 3], 'VOTING__DT__min_samples_split':[2,10,20], 
          'VOTING__DT__min_samples_leaf':[1,10,20],'VOTING__LR__C':np.linspace(0.001, 3, 5)}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='roc_auc',verbose=3)
gcv.fit(X, y)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check

[CV 1/5] END VOTING__DT__max_depth=None, VOTING__DT__min_samples_leaf=1, VOTING__DT__min_samples_split=2, VOTING__LR__C=0.001;, score=0.981 total time=   0.1s
[CV 2/5] END VOTING__DT__max_depth=None, VOTING__DT__min_samples_leaf=1, VOTING__DT__min_samples_split=2, VOTING__LR__C=0.001;, score=0.979 total time=   0.0s


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check

[CV 3/5] END VOTING__DT__max_depth=None, VOTING__DT__min_samples_leaf=1, VOTING__DT__min_samples_split=2, VOTING__LR__C=0.001;, score=0.976 total time=   0.0s
[CV 4/5] END VOTING__DT__max_depth=None, VOTING__DT__min_samples_leaf=1, VOTING__DT__min_samples_split=2, VOTING__LR__C=0.001;, score=0.981 total time=   0.0s


...

[CV 5/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=2.25025;, score=0.948 total time=   0.0s
[CV 1/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=3.0;, score=0.948 total time=   0.0s
[CV 2/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=3.0;, score=0.945 total time=   0.0s


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check

[CV 3/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=3.0;, score=0.947 total time=   0.0s
[CV 4/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=3.0;, score=0.943 total time=   0.0s
[CV 5/5] END VOTING__DT__max_depth=3, VOTING__DT__min_samples_leaf=20, VOTING__DT__min_samples_split=20, VOTING__LR__C=3.0;, score=0.948 total time=   0.0s


C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Python\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001E9C238FE90>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_...
                                                                                             random_state=24)),
                                                                     ('LR',
                                                                      LogisticRegression(C=0.4,
                                                                                         random_state=24)),
                                                                     ('NB',
                                                                      GaussianNB())],
                                                         voting='soft'))]),
             param_grid={'VOTING__DT__max_depth': [None, 2, 3],
                         'VOTING__DT__min_samples_leaf': [1, 10, 20],
                         'VOTING__DT__min_samples_split': [2, 10, 20],
                         'VOTING__LR__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00])},
             scoring='roc_auc', verbose=3)

In [44]:
print(gcv.best_params_)
print(gcv.best_score_)

{'VOTING__DT__max_depth': None, 'VOTING__DT__min_samples_leaf': 1, 'VOTING__DT__min_samples_split': 2, 'VOTING__LR__C': 0.001}
0.980099939210405
